<a href="https://colab.research.google.com/github/polychrome7/GCI2020/blob/main/week5%E5%AE%BF%E9%A1%8C_1110%E6%8F%90%E5%87%BA%E7%B7%A0%E5%88%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework3
下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額を算出し、さらに金額をベースに降順に並び替えをします。  
3. カラムが`CustomerID`と合計金額のあるテーブルを作成してください。  
4. そこから、購買者を合計金額をベースに$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数）にグループ分けします。  その際、合計金額が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とします。
5. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
6. それぞれのグループが売上の何割を占めるかを求めてください。その際、「グループ1」〜「グループn」の順に並べ、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出場所はOmnicampus右上にある青字のHomeworkドロップダウンメニューからSaveしてください。Solutionに関数を貼り付けてからSaveを押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [1]:
# common
import pandas as pd
import numpy as np

# init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [4]:
target_online_retail_data_tb

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60


In [110]:
# working place. everything 
def homework(target_online_retail_data_tb, n):
  df = target_online_retail_data_tb
  df_select = df[['CustomerID', 'TotalPrice']].dropna()
  df1=df_select.groupby('CustomerID',as_index=False)['TotalPrice'].sum()
  df1_sort=df1.sort_values(by='TotalPrice', ascending=False, ignore_index=True)
  s= [i for i in range(n)]
  index=pd.Series(s)
  total_qcut= df1_sort.groupby(pd.qcut(df1_sort['TotalPrice'], n, labels = index))
  df2 = []
  for cls, data in total_qcut:
    data = pd.DataFrame(data)
    data['group'] = cls
    df2.append(data)
  df2 = pd.concat(df2)
  result=df2.groupby(df2['group'].sort_values(ascending=True))['TotalPrice'].sum()/df2['TotalPrice'].sum()
  result1=result.sort_values(ascending=False)
  my_result1=pd.Series(result1)
  my_result=my_result1.values.tolist()
    
  return my_result


In [109]:
my_result


[0.6547716437018787,
 0.13652258578062726,
 0.07972795069576918,
 0.05065496113104422,
 0.03411429532515765,
 0.022424116722375965,
 0.014592842275980743,
 0.00719160436716555]